# Chapter 4. Company to Symbol Linking
This notebook is complementary material to Chapter 4. of the thesis.

## Naive Approach



- For named entity recognition we will use [spaCy](https://spacy.io/) library.


In [ ]:
%pip install spacy
%pip install spacy-transformers

- [List of spaCy english trained models](https://spacy.io/models/en)
- Due to state of art performance, we will use `en_core_web_trf` model.


In [ ]:
!python3 -m spacy download en_core_web_trf

In [1]:
import spacy
from spacy import displacy

# Load the transformer-based model
nlp = spacy.load("en_core_web_trf")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- We will use the excerpt from Guardian's technology [article](https://www.theguardian.com/uk-news/2024/feb/25/uks-enemies-could-use-ai-deepfakes-to-try-to-rig-election-says-james-cleverly).


In [2]:
# The excerpt from the article
text = "Executives from Adobe, Amazon, Google, IBM, Meta, Microsoft, OpenAI and TikTok gathered at the Munich Security Conference to announce a new framework for how they will respond to AI-generated deepfakes that deliberately trick voters."

# Load the article's text into the model
doc = nlp(text)

# Get collection of entities
entities = doc.ents

# Render the entities in the text
displacy.render(doc, style="ent", jupyter=True)

In [3]:
# Print the count of found entities
print(f"Entities count: {len(entities)}")

# Print the entities with their class labels
print("\nAll entities:")
print(" <entity> (<label>)")
for entity in entities:
    print(f" {entity.text} ({entity.label_})")

Entities count: 9

All entities:
 <entity> (<label>)
 Adobe (ORG)
 Amazon (ORG)
 Google (ORG)
 IBM (ORG)
 Meta (ORG)
 Microsoft (ORG)
 OpenAI (ORG)
 TikTok (ORG)
 the Munich Security Conference (EVENT)


In [4]:
# Filter entities to only include ORG entities
org_entities = [entity for entity in entities if entity.label_ == "ORG"]

# Print the count of ORG entities
print(f"\nORG entities count: {len(org_entities)}")
print("\nORG entities:")
print(" <entity>")
for entity in org_entities:
    print(f" {entity.text}")


ORG entities count: 8

ORG entities:
 <entity>
 Adobe
 Amazon
 Google
 IBM
 Meta
 Microsoft
 OpenAI
 TikTok


### Dataset Preparation

- We will use the following datasets `NASDAQ.csv`,`NYSE.csv` and `AMEX.csv` for company to symbol matching.
- The datasets are downloaded from [eoddata](https://eoddata.com/) and are stored in `/data/symbols` directory.
- The datasets contain the list of comapnies and their symbols for the respective exchanges.


In [ ]:
%pip install pandas

In [5]:
import pandas as pd
import csv

# For google colab use the path "/content/data/symbols/<exchange>.csv"
# For local use the path "data/symbols/<exchange>.csv"
# Read the CSV file of without considering quotes
nasdaq_df = pd.read_csv(
    "data/symbols/NASDAQ.csv", sep="\t", quoting=csv.QUOTE_NONE, header=0
)

nyse_df = pd.read_csv(
    "data/symbols/NYSE.csv", sep="\t", quoting=csv.QUOTE_NONE, header=0
)

amex_df = pd.read_csv(
    "data/symbols/AMEX.csv", sep="\t", quoting=csv.QUOTE_NONE, header=0
)

# Remove quotes from each column
nasdaq_df = nasdaq_df.apply(lambda x: x.str.replace('"', ""))
nyse_df = nyse_df.apply(lambda x: x.str.replace('"', ""))
amex_df = amex_df.apply(lambda x: x.str.replace('"', ""))

# Remove quotes from the column names
nasdaq_df.columns = nasdaq_df.columns.str.replace('"', "")
nyse_df.columns = nyse_df.columns.str.replace('"', "")
amex_df.columns = amex_df.columns.str.replace('"', "")

# Rename the columns
nasdaq_df.rename(columns={"Symbol": "ticker", "Description": "name"}, inplace=True)
nyse_df.rename(columns={"Symbol": "ticker", "Description": "name"}, inplace=True)
amex_df.rename(columns={"Symbol": "ticker", "Description": "name"}, inplace=True)

print("NASDAQ")
display(nasdaq_df.head())
display(nasdaq_df.count())
print("\n")

print("NYSE")
display(nyse_df.head())
display(nyse_df.count())
print("\n")

print("AMEX")
display(amex_df.head())
display(amex_df.count())
print("\n")

NASDAQ


,ticker,name
0,AACG,Ata Creativity Global ADR
1,AACI,Armada Acquisition Corp I
2,AACIU,Armada Acquisition Corp I
3,AACIW,Armada Acquisition Corp I WT
4,AADI,Aadi Biosciences Inc


ticker    5073
name      5073
dtype: int64



NYSE


,ticker,name
0,A,Agilent Technologies
1,AA,Alcoa Corp
2,AAC,Ares Acquisition Corp Cl A
3,AAC.S,Ares Acquisition Corp II [Aact.U]
4,AAC.T,Ares Acquisition Corp II [Aact.Ws]


ticker    3102
name      3101
dtype: int64



AMEX


,ticker,name
0,AAA,First Priority Clo Bond ETF
1,AAAU,GS Physical Gold ETF
2,AAMC,Altisource Asset Management Corp Com
3,AAPX,T-Rex 2X Long Apple Daily Target ETF
4,AAPY,Neos Strategy Apple [Aapl] ETF


ticker    3165
name      3165
dtype: int64

In [6]:
# If columns count in each dataset is not equal, then ensure which records are uncomplete and delete them


# NASDAQ
print("NASDAQ")
display(nasdaq_df[nasdaq_df.isnull().any(axis=1)])

# Drop the rows with missing values
nasdaq_df.dropna(inplace=True)
print("\n")


# NYSE
print("NYSE")
display(nyse_df[nyse_df.isnull().any(axis=1)])

# Drop the rows with missing values
nyse_df.dropna(inplace=True)
print("\n")


# AMEX
print("AMEX")
display(amex_df[amex_df.isnull().any(axis=1)])

# Drop the rows with missing values
amex_df.dropna(inplace=True)
print("\n")

NASDAQ


,ticker,name




NYSE


,ticker,name
2039,NTEST.I,NaN




AMEX


,ticker,name


In [7]:
# Concatenate two dataframes into a single one with ticker alphabetical order
tickers_df = pd.concat([nasdaq_df, nyse_df, amex_df]).sort_values(by="ticker")

# Reset the index
tickers_df.reset_index(drop=True, inplace=True)

display(tickers_df)

,ticker,name
0,A,Agilent Technologies
1,AA,Alcoa Corp
2,AAA,First Priority Clo Bond ETF
3,AAAU,GS Physical Gold ETF
4,AAC,Ares Acquisition Corp Cl A
...,...,...
11334,ZVRA,Zevra Therapeutics Inc
11335,ZVSA,Zyversa Therapeutics Inc
11336,ZWS,Zurn Elkay Water Solutions Corp
11337,ZYME,Zymeworks Inc


### Name Matching library

- Python fuzzy matching library for company name matching.
- Source: https://github.com/DeNederlandscheBank/name_matching

In [ ]:
%pip install name_matching

*Note: At the beggining of this project, I want to define own Matcher class to easier use in backend architecture of application.*

In [8]:
from name_matching.name_matcher import NameMatcher
import pandas as pd
from typing import List, Dict

tickers_df_copy = tickers_df

class Matcher:
    """
    A class to match company names to tickers.
    """

    def __init__(
        self,
        lowercase: bool,
        punctuations: bool,
        remove_ascii: bool,
        legal_suffixes: bool,
        common_words: bool,
        verbose: bool,
        distance_metrics: List[str],
        MIN_SCORE: int = 80,
    ):
        """
        The constructor for Matcher class.

        Args:
            lowercase (bool): Reffering to the lowercase in the NameMatcher class.
            punctuations (bool): Reffering to the punctuations in the NameMatcher class.
            remove_ascii (bool): Reffering to the remove_ascii in the NameMatcher class.
            legal_suffixes (bool): Reffering to the legal_suffixes in the NameMatcher class.
            common_words (bool): Reffering to the common_words in the NameMatcher class.
            verbose (bool): Reffering to the verbose in the NameMatcher class.
            distance_metrics (List[str]): Reffering to the distance_metrics in the NameMatcher class.
            MIN_SCORE (int, optional): The minimum score for a match. Defaults to 80.
        """
        self.min_score = MIN_SCORE
        self.instance = NameMatcher(
            number_of_matches=1,
            top_n=1,
            lowercase=lowercase,
            punctuations=punctuations,
            remove_ascii=remove_ascii,
            legal_suffixes=legal_suffixes,
            common_words=common_words,
            verbose=verbose,
        )

        self.instance.set_distance_metrics(distance_metrics)

        self.instance.load_and_process_master_data(
            column="name", 
            df_matching_data=tickers_df,
            transform=True)

    def match_company_name(self, company_names: List[str]) -> pd.DataFrame:
        """
        Matches company names extracted from the article to the exchange dataset.

        Args:
            company_names (List[str]): The list of article's company names to match.
        Returns:
            pd.DataFrame: The DataFrame containing the matches.
        """
        
        # Convert the list of company names to a DataFrame
        company_names_df = pd.DataFrame(company_names, columns=["original_name"])

        # Match the company names
        matches_df = self.instance.match_names(
            to_be_matched=company_names_df, column_matching="original_name"
        )

        return matches_df

    def postprocess_matches(self, matches_df: pd.DataFrame) -> Dict[str, str]:
        """
        Postprocesses the matches DataFrame and returns a dictionary with company names as keys and tickers as values.
        Records with score below the minimum score are filtered out.

        Args:
            matches_df (pd.DataFrame): The DataFrame containing the matches.

        Returns:
            dict: A dictionary with company names as keys and tickers as values.
        """
        matches_df = matches_df[matches_df["score"] > self.min_score]
        matches_df = matches_df.drop(columns=["score", "match_name"])
        matches_df.loc[:, "ticker"] = matches_df["match_index"].apply(
            lambda x: tickers_df.iloc[x]["ticker"]
        )
        matches_dict = matches_df.set_index("original_name")["ticker"].to_dict()

        return matches_dict

In [9]:
a_matcher = Matcher(
    lowercase=True,
    punctuations=True,
    remove_ascii=True,
    legal_suffixes=True,
    common_words=True,
    verbose=False,
    distance_metrics=["discounted_levenshtein"],
    MIN_SCORE=80,
)

a_matches_raw_result = a_matcher.match_company_name(org_entities)

# Combine the matches with the tickers DataFrame
combined_df = pd.merge(a_matches_raw_result, tickers_df_copy, left_on="match_index", right_index=True)

# Drop the match_index column
combined_df.drop(columns=["match_index"], inplace=True)

display("Discounted Levenshtein", combined_df)

# Postprocess the matches to get the final result according to the minimum score
a_matches = a_matcher.postprocess_matches(a_matches_raw_result)
display(a_matches)

# Do not be misselading with the match_name column, read description of legal_suffixes in the NameMatcher class

'Discounted Levenshtein'

,original_name,match_name,score,ticker,name
0,adobe,adobe systems inc,100.000000,ADBE,adobe systems inc
1,amazon,amazoncom inc,74.450584,AMZN,amazoncom inc
2,google,neos yield premium strategy google googl etf,35.917230,GOOP,neos yield premium strategy google googl etf
3,ibm,ibio inc,55.647806,IBIO,ibio inc
4,meta,kennametal inc,37.796421,KMT,kennametal inc
5,microsoft,microsoft corp,100.000000,MSFT,microsoft corp
6,openai,open bank,73.286122,OPBK,open bank
7,tiktok,cytokinetics,27.436368,CYTK,cytokinetics


{'adobe': 'ADBE', 'microsoft': 'MSFT'}

In [10]:
b_matcher = Matcher(
    lowercase=True,
    punctuations=True,
    remove_ascii=True,
    legal_suffixes=True,
    common_words=True,
    verbose=False,
    distance_metrics=["weighted_jaccard"],
    MIN_SCORE=80,
)

b_matches_raw_result = b_matcher.match_company_name(org_entities)

# Combine the matches with the tickers DataFrame
combined_df = pd.merge(b_matches_raw_result, tickers_df_copy, left_on="match_index", right_index=True)

# Drop the match_index column
combined_df.drop(columns=["match_index"], inplace=True)

display("Weighted Jaccard", combined_df)

# Postprocess the matches to get the final result according to the minimum score
a_matches = a_matcher.postprocess_matches(b_matches_raw_result)
display(a_matches)

# Do not be misselading with the match_name column, read description of legal_suffixes in the NameMatcher class

'Weighted Jaccard'

,original_name,match_name,score,ticker,name
0,adobe,adobe systems inc,100.000000,ADBE,adobe systems inc
1,amazon,amazoncom inc,78.260870,AMZN,amazoncom inc
2,google,neos yield premium strategy google googl etf,50.000000,GOOP,neos yield premium strategy google googl etf
3,ibm,ibio inc,54.545455,IBIO,ibio inc
4,meta,kennametal inc,47.368421,KMT,kennametal inc
5,microsoft,microsoft corp,100.000000,MSFT,microsoft corp
6,openai,open bank,75.000000,OPBK,open bank
7,tiktok,cytokinetics,39.130435,CYTK,cytokinetics


{'adobe': 'ADBE', 'microsoft': 'MSFT'}

In [11]:
c_matcher = Matcher(
    lowercase=True,
    punctuations=True,
    remove_ascii=True,
    legal_suffixes=True,
    common_words=True,
    verbose=False,
    distance_metrics=["fuzzy_wuzzy_token_set"],
    MIN_SCORE=80,
)

c_matches_raw_result = c_matcher.match_company_name(org_entities)


# Combine the matches with the tickers DataFrame
combined_df = pd.merge(c_matches_raw_result, tickers_df_copy, left_on="match_index", right_index=True)

# Drop the match_index column
combined_df.drop(columns=["match_index"], inplace=True)

display("Fuzzy Wuzzy Token Set", combined_df)

# Postprocess the matches to get the final result according to the minimum score
c_matches = c_matcher.postprocess_matches(c_matches_raw_result)
display(c_matches)

'Fuzzy Wuzzy Token Set'

,original_name,match_name,score,ticker,name
0,adobe,adobe systems inc,100.000000,ADBE,adobe systems inc
1,amazon,amazoncom inc,82.352941,AMZN,amazoncom inc
2,google,neos yield premium strategy google googl etf,100.000000,GOOP,neos yield premium strategy google googl etf
3,ibm,ibio inc,66.666667,IBIO,ibio inc
4,meta,kennametal inc,62.500000,KMT,kennametal inc
5,microsoft,microsoft corp,100.000000,MSFT,microsoft corp
6,openai,open bank,83.333333,OPBK,open bank
7,tiktok,cytokinetics,40.000000,CYTK,cytokinetics


{'adobe': 'ADBE',
 'amazon': 'AMZN',
 'google': 'GOOP',
 'microsoft': 'MSFT',
 'openai': 'OPBK'}

*Sample presented on Name Matcher with results copied from github page for better understanding of the library.*

In [12]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

# define a dataset with bank names
df_companies_a = pd.DataFrame({'Company name': [
        'Industrial and Commercial Bank of China Limited',
        'China Construction Bank',
        'Agricultural Bank of China',
        'Bank of China',
        'JPMorgan Chase',
        'Mitsubishi UFJ Financial Group',
        'Bank of America',
        'HSBC',
        'BNP Paribas',
        'Crédit Agricole']})

# alter each of the bank names a bit to test the matching
df_companies_b = pd.DataFrame({'name': [
        'Bank of China Limited',
        'Mitsubishi Financial Group',
        'Construction Bank China',
        'Agricultural Bank',
        'Bank of Amerika',
        'BNP Parisbas',
        'JP Morgan Chase',
        'HSCB',
        'Industrial and Commercial Bank of China',
        'Credite Agricole']})

# initialise the name matcher
matcher = NameMatcher(number_of_matches=1, 
                      legal_suffixes=True, 
                      common_words=False, 
                      top_n=50, 
                      verbose=True)

# adjust the distance metrics to use
matcher.set_distance_metrics(['bag', 'typo', 'refined_soundex'])

# load the data to which the names should be matched
matcher.load_and_process_master_data(column='Company name',
                                     df_matching_data=df_companies_a, 
                                     transform=True)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=df_companies_b, 
                              column_matching='name')

# combine the datasets based on the matches
combined = pd.merge(df_companies_a, matches, how='left', left_index=True, right_on='match_index')
combined = pd.merge(combined, df_companies_b, how='left', left_index=True, right_index=True)

display(combined)


preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████| 1/1 [00:00<00:00, 1440.85it/s]


possible matches found   
 fuzzy matching...



100%|██████████| 10/10 [00:03<00:00,  2.66it/s]

done


,Company name,original_name,match_name,score,match_index,name
8,industrial and commercial bank of china limited,industrial and commercial bank of china,industrial and commercial bank of china limited,100.000000,0,industrial and commercial bank of china
2,china construction bank,construction bank china,china construction bank,74.916388,1,construction bank china
3,agricultural bank of china,agricultural bank,agricultural bank of china,69.780220,2,agricultural bank
0,bank of china,bank of china limited,bank of china,100.000000,3,bank of china limited
6,jpmorgan chase,jp morgan chase,jpmorgan chase,95.555556,4,jp morgan chase
1,mitsubishi ufj financial group,mitsubishi financial group,mitsubishi ufj financial group,86.666667,5,mitsubishi financial group
4,bank of america,bank of amerika,bank of america,93.333333,6,bank of amerika
7,hsbc,hscb,hsbc,75.000000,7,hscb
5,bnp paribas,bnp parisbas,bnp paribas,89.682540,8,bnp parisbas
9,credit agricole,credite agricole,credit agricole,95.833333,9,credite agricole
